In [1]:
# # Building Name & State Dimensions & Control Dimensions & Num. of Constraints & Main Objective (Minimize) \\
# from energym.wrappers.rl_wrapper import StableBaselinesRLWrapper
# from buildings_factory import *
# objectives = ["average exchanged energy",
#               "average exchanged energy",
#               "average exchanged energy",
#               "average exchanged energy",
#               "power demand",
#               "power demand",
#               "CO2 emission",
#               "CO2 emission",
#               "power demand",
#               "power demand",
#               "power demand",
#               "power demand"]

# model_info = ""
# for building_name in buildings_list:
#     # min_kpis, max_kpis = collect_baseline_kpi(building_name)
#     building_idx = buildings_list.index(building_name)
#     env = get_env(building_name)
#     inputs = get_inputs(building_name, env)
#     default_control = default_controls[building_idx]
#     env_rl = StableBaselinesRLWrapper(building_name, None, None, reward_func)
#     input_dim = env_rl.observation_space.shape[0]
#     control_dim = env_rl.action_space.shape[0]
#     constraint_num = 0
#     for key, val in env.kpi_options.items():
#         if val["type"] == 'avg_dev': constraint_num += 1 
#     main_objective = objectives[building_idx]
#     model_info += f"{building_name} & {input_dim} & {control_dim} & {constraint_num} & {main_objective} \\ "
# print(model_info)

In [2]:
import pandas as pd
# buildings_list = ["ApartmentsThermal-v0", "ApartmentsGrid-v0", "Apartments2Thermal-v0",
#                   "Apartments2Grid-v0", "OfficesThermostat-v0", "MixedUseFanFCU-v0",
#                   "SeminarcenterThermostat-v0", "SeminarcenterFull-v0", "SimpleHouseRad-v0",
#                   "SimpleHouseRSla-v0", "SwissHouseRSlaW2W-v0", "SwissHouseRSlaTank-v0"] 

plot_building_names = ["SimpleHouseRad-v0"]
seeds = [7,13]
# "MOPO_ori_simulator"
model_names = ["TD3PlusBC_ori_simulator", "CQL_ori_simulator", "MOPO_ori_simulator", "SAC_bs_predictor", "SAC_dnn_predictor"]
legend_names = ["TD3PlusBC", "CQL", "MOPO", "Best", "Ours"]
# model_names = ["TD3PlusBC_ori_simulator", "CQL_ori_simulator", "SAC_bs_predictor", "SAC_dnn_predictor"]
# legend_names = ["TD3PlusBC", "CQL", "Best", "Ours"]
res_cols = ["manual_baseline_eval_episode_reward", "manual_eval_episode_reward"]
            
colors = ['r', 'g', 'r--', 'g--']

res_data = []
for seed in seeds:
    for model_name, legend_name in zip(model_names, legend_names):
        for building in plot_building_names:
            res_loc = f"data/models/{building}/{model_name}_seed{seed}/logs/best/rewards.csv"
            res_row = pd.read_csv(res_loc).iloc[-1, :]
            res_data.append([seed, building, model_name, legend_name,
                             res_row["eval_episode_reward"], res_row["manual_eval_episode_reward"],
                             res_row["baseline_eval_episode_reward"], res_row["manual_baseline_eval_episode_reward"]])
res_df = pd.DataFrame(res_data, columns=["seed", "building", "model_name", 
                                         "legend", "learnt_reward", "manual_reward",
                                         "learnt_baseline_reward", "manual_baseline_reward"])
            

res_df.head()

,seed,building,model_name,legend,learnt_reward,manual_reward,learnt_baseline_reward,manual_baseline_reward
0,7,SimpleHouseRad-v0,TD3PlusBC_ori_simulator,TD3PlusBC,-20482.469267,-20482.469267,-11657.800858,-11657.800858
1,7,SimpleHouseRad-v0,CQL_ori_simulator,CQL,-5905.805732,-5905.805732,-11658.130906,-11658.130906
2,7,SimpleHouseRad-v0,MOPO_ori_simulator,MOPO,-20631.857304,-20631.857304,-11658.181170,-11658.181170
3,7,SimpleHouseRad-v0,SAC_bs_predictor,Best,95636.966998,-6826.075410,81688.277918,-11658.181233
4,7,SimpleHouseRad-v0,SAC_dnn_predictor,Ours,48493.333834,-8100.186446,42209.973207,-11657.475607


In [3]:
import numpy as np
cols = res_df.columns.tolist()
# "RL_reward", "manual_RL_reward", "baseline_reward", "manual_baseline_reward"
cols.remove("seed")
stat_df = res_df[cols].groupby(["building", "model_name", "legend"]).agg(learnt_reward_mean=("learnt_reward", np.mean),
                                                           learnt_reward_std=("learnt_reward", np.std),
                                                           learnt_baseline_reward_mean=("learnt_baseline_reward", np.mean),
                                                           learnt_baseline_reward_std=("learnt_baseline_reward", np.std),
                                                           manual_reward_mean=("manual_reward", np.mean),
                                                           manual_reward_std=("manual_reward", np.std),
                                                           manual_baseline_reward_mean=("manual_baseline_reward", np.mean),
                                                           manual_baseline_reward_std=("manual_baseline_reward", np.std)).reset_index()

stat_df.fillna(0.0, inplace=True)
stat_df

,building,model_name,legend,learnt_reward_mean,learnt_reward_std,learnt_baseline_reward_mean,learnt_baseline_reward_std,manual_reward_mean,manual_reward_std,manual_baseline_reward_mean,manual_baseline_reward_std
0,SimpleHouseRad-v0,CQL_ori_simulator,CQL,-10069.986915,5889.041504,-11658.115994,0.021088,-10069.986915,5889.041504,-11658.115994,0.021088
1,SimpleHouseRad-v0,MOPO_ori_simulator,MOPO,-20633.602157,2.467595,-11658.178714,0.003473,-20633.602157,2.467595,-11658.178714,0.003473
2,SimpleHouseRad-v0,SAC_bs_predictor,Best,95793.240402,221.003967,81688.440656,0.230146,-6359.616204,659.672935,-11658.086833,0.133503
3,SimpleHouseRad-v0,SAC_dnn_predictor,Ours,48962.138663,662.990148,42204.310079,8.008872,-8347.237622,349.383123,-11657.973846,0.704617
4,SimpleHouseRad-v0,TD3PlusBC_ori_simulator,TD3PlusBC,-20079.879555,569.347830,-11658.107298,0.433370,-20079.879555,569.347830,-11658.107298,0.433370


In [4]:
bs_stat_df = stat_df[['building', "manual_baseline_reward_mean"]].groupby("building").mean().reset_index()
bs_stat_df.columns = ['building', 'manual_reward_mean']
bs_stat_df['legend'] = 'baseline'
bs_stat_df['manual_reward_std'] = 0.0

In [5]:
cols = ["building", "legend", "manual_reward_mean", "manual_reward_std"]
stat_df_all = pd.concat([stat_df[cols], bs_stat_df[cols]])
stat_df_all['reward'] = stat_df_all.apply(lambda x: f"{x['manual_reward_mean']:.2f}\xb1{x['manual_reward_std']}", axis=1)
df = stat_df_all.pivot(index='building', columns='legend', values='reward').reset_index()
df

legend,building,Best,CQL,MOPO,Ours,TD3PlusBC,baseline
0,SimpleHouseRad-v0,-6359.62±659.6729351282808,-10069.99±5889.041503967595,-20633.60±2.4675951031174836,-8347.24±349.38312346559377,-20079.88±569.3478303066163,-11658.09±0.0


In [6]:
print_cols = ["building", "baseline", "CQL", "TD3PlusBC", "MOPO", "Ours", "Best"]
print(" & ".join(print_cols))
for _, row in df.iterrows():
    print(" & ".join([str(row[col]) for col in print_cols]))

building & baseline & CQL & TD3PlusBC & MOPO & Ours & Best
SimpleHouseRad-v0 & -11658.09±0.0 & -10069.99±5889.041503967595 & -20079.88±569.3478303066163 & -20633.60±2.4675951031174836 & -8347.24±349.38312346559377 & -6359.62±659.6729351282808
